In [21]:
import pandas as pd
import torch
import dgl
from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler
import networkx as nx
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv
import time
import math


In [22]:
nodes_df = pd.read_csv('clean_nodes.csv')
edges_df = pd.read_csv('clean_edges.csv')

1.Xây dựng đồ thị
# Tạo mapping từ profile_id sang index (0 đến N-1)
# Tạo tensor cho edge_index (dạng (2, num_edges))
# Tạo đồ thị DGL

In [23]:
profile_ids = nodes_df['profile_id'].tolist()
node_to_idx = {user: i for i, user in enumerate(profile_ids)}
# 2. Chuyển đổi danh sách cạnh thành tensor edge_index cho DGL
edge_list = []
for idx, row in edges_df.iterrows():
    src, tgt = row['source'], row['target']
    if src in node_to_idx and tgt in node_to_idx:
        edge_list.append( (node_to_idx[src], node_to_idx[tgt]) )


In [24]:
# Chuyển đổi edge_list thành tensor, với định dạng (source_indices, target_indices)
src_nodes, tgt_nodes = zip(*edge_list)
edge_array = torch.tensor(edge_list, dtype=torch.long).t()  # Kích thước [2, num_edges]

# Tạo đồ thị DGL
g = dgl.graph((edge_array[0], edge_array[1]), num_nodes=len(profile_ids))
print("Số lượng nút:", g.num_nodes(), "Số lượng cạnh:", g.num_edges())

Số lượng nút: 25669 Số lượng cạnh: 53452


2. Tính toán các chỉ số mạng cơ bản

In [25]:
nx_g = dgl.to_networkx(g,node_attrs=g.ndata,edge_attrs=g.edata)
print(nx_g.is_directed())
num_nodes = g.num_nodes()
print("Đã chuyển sang NetworkX.")

def approximate_centrality_metrics(G, sample_ratio=0.1):
    num_nodes = G.number_of_nodes()
    
    # Chọn mẫu ngẫu nhiên các nút
    sampled_nodes = np.random.choice(
        list(G.nodes()), 
        size=int(num_nodes * sample_ratio), 
        replace=False
    )
    # In-degree Centrality
    in_degrees = nx.in_degree_centrality(G)
    # Out-degree Centrality
    out_degrees = nx.out_degree_centrality(G)
    print('completed in/out degree')
    # Betweenness Centrality (xấp xỉ)
    betweenness = nx.betweenness_centrality(
        G, 
        k=min(500, len(sampled_nodes)),  # Giới hạn số nút mẫu
        normalized=True
    )
    print('completed betweeness centrality')
    # reciprocity tính đối ứng
    # reciprocity = nx.reciprocity(G,G.nodes())
    # clossness
    # closeness = nx.closeness_centrality(G)
    # print('completed clossness centrality')
    # Eigenvector Centrality (xấp xỉ)
    try:
        eigenvector = nx.eigenvector_centrality(
            G, 
            max_iter=100,  # Giới hạn số lần lặp
            tol=1e-3       # Ngưỡng hội tụ
        )
    except:
        # Nếu không hội tụ, sử dụng degree centrality làm xấp xỉ
        eigenvector = {n: d for n, d in G.degree()}
        max_degree = max(eigenvector.values())
        eigenvector = {n: d/max_degree for n, d in eigenvector.items()}
    
    return {
        'in_degree': in_degrees,
        'out_degree': out_degrees,
        'betweenness': betweenness,
        # 'reciprocity': reciprocity,
        'eigenvector': eigenvector
    }

centrality_metrics = approximate_centrality_metrics(nx_g)
# Tạo DataFrame chứa các chỉ số mạng (sắp xếp theo index của node)
metrics_df = pd.DataFrame({
    'profile_id': profile_ids,
    'in_degree': [centrality_metrics['in_degree'].get(node, 0) for node in range(num_nodes)],
    'out_degree': [centrality_metrics['out_degree'].get(node, 0) for node in range(num_nodes)],
    'betweenness': [centrality_metrics['betweenness'].get(node, 0.0) for node in range(num_nodes)],
    # 'reciprocity': [centrality_metrics['reciprocity'].get(node, 0.0) for node in range(num_nodes)],
    # 'closeness': [centrality_metrics['closeness'].get(node, 0.0) for node in range(num_nodes)],
    'eigenvector': [centrality_metrics['eigenvector'].get(node, 0.0) for node in range(num_nodes)]
})
print("Các chỉ số mạng tính được, hiển thị 5 dòng đầu:")
print(metrics_df.head())

True
Đã chuyển sang NetworkX.
completed in/out degree
completed betweeness centrality
Các chỉ số mạng tính được, hiển thị 5 dòng đầu:
   profile_id  in_degree  out_degree  betweenness  eigenvector
0          14   0.002376    0.002104     0.008969     0.018924
1         512   0.000000    0.000039     0.000000     0.000165
2         533   0.000039    0.000039     0.000000     0.000329
3         534   0.126695    0.110059     0.164540     1.000000
4         564   0.004285    0.000117     0.000454     0.018595


# 3. Nhúng đồ thị sử dụng GNN với DGL (GCN)


In [26]:
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_feats)
        self.conv2 = GraphConv(hidden_feats, out_feats)
    
    def forward(self, graph, features):
        x = self.conv1(graph, features)
        x = F.relu(x)
        x = self.conv2(graph, x)
        return x

In [27]:
# Nếu chưa có đặc trưng cho nút, tạo feature ngẫu nhiên (ví dụ, kích thước 16)
feat_dim = 16
if 'feat' not in g.ndata:
    g.ndata['feat'] = torch.randn(g.num_nodes(), feat_dim)

embedding_dim = 16  # Kích thước vector nhúng mong muốn
model = GCN(feat_dim, 2 * embedding_dim, embedding_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


In [28]:
# Để giảm chi phí tính toán, ta huấn luyện số epoch thấp (ví dụ 100 epoch)
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    g = dgl.add_self_loop(g)
    # Ở đây, làm bài toán tự giám sát đơn giản: tái tạo lại đặc trưng ban đầu bằng MSE
    z = model(g, g.ndata['feat'])
    loss = F.mse_loss(z, g.ndata['feat'])
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:03d}: Loss = {loss.item():.4f}")

# Sau huấn luyện, trích xuất vector nhúng của các nút
model.eval()
with torch.no_grad():
    node_embeddings = model(g, g.ndata['feat']).cpu().numpy()

# Tạo DataFrame cho embedding: đặt tên cột là gnn_emb_0, gnn_emb_1, ... gnn_emb_63
embedding_df = pd.DataFrame(node_embeddings, columns=[f'gnn_emb_{i}' for i in range(embedding_dim)])
embedding_df['profile_id'] = profile_ids

Epoch 010: Loss = 1.3111
Epoch 020: Loss = 1.2123
Epoch 030: Loss = 1.1001
Epoch 040: Loss = 0.9979
Epoch 050: Loss = 0.9071
Epoch 060: Loss = 0.8264
Epoch 070: Loss = 0.7541
Epoch 080: Loss = 0.6889
Epoch 090: Loss = 0.6298
Epoch 100: Loss = 0.5762


In [29]:
##############################################
# 4. Kết hợp đặc trưng: Metrics + GNN Embedding
##############################################
# Chuẩn hóa các chỉ số centrality
scaler_centrality = MinMaxScaler()
centrality_columns = ['in_degree', 'out_degree', 'betweenness', 'eigenvector']
metrics_df[centrality_columns] = scaler_centrality.fit_transform(metrics_df[centrality_columns])

# Chuẩn hóa node embeddings
scaler_embedding = StandardScaler()
embedding_columns = [f'gnn_emb_{i}' for i in range(embedding_dim)]
embedding_df[embedding_columns] = scaler_embedding.fit_transform(embedding_df[embedding_columns])

# Phần code còn lại giữ nguyên
combined_df = pd.merge(metrics_df, embedding_df, on='profile_id')
print("Dữ liệu sau khi chuẩn hóa với RobustScaler:")
print(combined_df.describe())

# Lưu kết quả ra file CSV
combined_df.to_csv('combined_user_features.csv', index=False)
print("Đã lưu đặc trưng kết hợp vào file 'combined_user_features.csv'")

Dữ liệu sau khi chuẩn hóa với RobustScaler:
         profile_id     in_degree    out_degree   betweenness   eigenvector  \
count  2.566900e+04  25669.000000  25669.000000  25669.000000  25669.000000   
mean   3.633382e+06      0.000640      0.000737      0.000301      0.000521   
std    3.134120e+06      0.010569      0.009013      0.009077      0.009414   
min    1.400000e+01      0.000000      0.000000      0.000000      0.000000   
25%    1.013983e+06      0.000000      0.000000      0.000000      0.000000   
50%    2.815261e+06      0.000308      0.000354      0.000000      0.000000   
75%    5.824166e+06      0.000308      0.000354      0.000000      0.000165   
max    1.131592e+07      1.000000      1.000000      1.000000      1.000000   

          gnn_emb_0     gnn_emb_1     gnn_emb_2     gnn_emb_3     gnn_emb_4  \
count  2.566900e+04  2.566900e+04  2.566900e+04  2.566900e+04  2.566900e+04   
mean   1.605000e-08 -6.241665e-09 -1.070000e-08 -8.173608e-09  6.836109e-09   
std    